In [1]:
!pip install pandas

In [ ]:
import pandas as pd
import ee
from datetime import timedelta

# Khởi tạo Google Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-nguyendangkhoi9517')

# Đọc file data.csv
df = pd.read_csv("GLC03122015_20241006.csv")

# Chuyển đổi định dạng ngày từ "M/D/YY" sang "YYYY-MM-DD"
df['dfo_began'] = pd.to_datetime(df['dfo_began'], format='%m/%d/%y', errors='coerce').dt.strftime('%Y-%m-%d')
df['dfo_ended'] = pd.to_datetime(df['dfo_ended'], format='%m/%d/%y', errors='coerce').dt.strftime('%Y-%m-%d')

# Kiểm tra dữ liệu sau khi chuyển đổi, thay thế null bằng 'unknown' hoặc loại bỏ các dòng có giá trị null
df.dropna(subset=['dfo_began', 'dfo_ended'], inplace=True)

# Hàm lấy lượng mưa từ CHIRPS
def get_precipitation(lon, lat, start_date, end_date):
    start_date = ee.Date(start_date)
    end_date = ee.Date(end_date)

    chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY") \
                .filterDate(start_date, end_date) \
                .filterBounds(ee.Geometry.Point([lon, lat]))

    precipitation = chirps.sum().reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=ee.Geometry.Point([lon, lat]),
        scale=5000
    ).get("precipitation")

    return precipitation.getInfo()

# Hàm tính các khoảng thời gian mưa
def get_precip_for_period(lon, lat, start_date, period_days):
    end_date = pd.to_datetime(start_date) + timedelta(days=period_days)
    end_date = end_date.strftime('%Y-%m-%d')
    try:
        precip = get_precipitation(lon, lat, start_date, end_date)
        return precip if precip is not None else 0
    except Exception as e:
        print(f"Error fetching data for ({lon}, {lat}) from {start_date} to {end_date}: {e}")
        return None

# Tạo danh sách lưu dữ liệu
rainfall_data = []

# Duyệt qua từng hàng của DataFrame
for _, row in df.iterrows():
    lon = row['dfo_centroid_x']
    lat = row['dfo_centroid_y']
    start_date = row['dfo_began']

    # Lấy lượng mưa cho các khoảng thời gian khác nhau
    precipitation_1month = get_precip_for_period(lon, lat, start_date, 30)  # 1 tháng (30 ngày)
    precipitation_2weeks = get_precip_for_period(lon, lat, start_date, 14)  # 2 tuần
    precipitation_1week = get_precip_for_period(lon, lat, start_date, 7)    # 1 tuần
    precipitation_3days = get_precip_for_period(lon, lat, start_date, 3)    # 3 ngày
    precipitation_1day = get_precip_for_period(lon, lat, start_date, 1)     # 1 ngày

    rainfall_data.append({
        'start_date': start_date,
        'longitude': lon,
        'latitude': lat,
        'precipitation_1month': precipitation_1month,
        'precipitation_2weeks': precipitation_2weeks,
        'precipitation_1week': precipitation_1week,
        'precipitation_3days': precipitation_3days,
        'precipitation_1day': precipitation_1day
    })

# Tạo DataFrame mới từ danh sách
rainfall_df = pd.DataFrame(rainfall_data)

# Hiển thị DataFrame kết quả
print(rainfall_df.head())

# Lưu DataFrame mới ra file CSV
rainfall_df.to_csv("rainfall_data.csv", index=False)

   start_date   longitude   latitude  precipitation_1month  \
0  2000-02-18  143.697800 -31.268059             49.118085   
1  2000-02-17   47.295670 -15.782624            292.474470   
2  2000-04-05   22.415404  46.763746             42.726614   
3  2000-07-11  105.063841  11.242567            130.655540   
4  2000-08-30  132.057679  43.773883            190.531867   

   precipitation_2weeks  precipitation_1week  precipitation_3days  \
0             20.752536            20.752536             7.679764   
1            190.112098           132.763813            84.012642   
2             30.538156            30.538156            23.372920   
3             74.468333            54.308254            21.060406   
4             81.418098            81.418098            81.418098   

   precipitation_1day  
0            0.000000  
1           33.605057  
2           10.889926  
3            0.000000  
4            0.000000  
